In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm import tqdm
import networkx as nx

from utils import projection_order1, mask_from_order
from data_generator import data_generator,generate_DAG, generator_matrix
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_dtype(torch.float64)
import opt
import re

In [2]:
seed = 0
torch.random.manual_seed(seed)
np.random.seed(seed)
B_scale = 1.0 
B_ranges = ((B_scale * -1.0, B_scale * -0.05), (B_scale * 0.05, B_scale * 1.0))
n_test = 10000
d = 100
degree = 4
graph_type = 'ER'
noise_type = 'gaussian_ev'
DAG = generate_DAG(d, graph_type, degree, B_ranges, hd = False, seed=seed)
assert nx.is_directed_acyclic_graph(nx.DiGraph(DAG.B))
B = torch.tensor(DAG.B).to(device)
GM = generator_matrix(B)
X_test = data_generator(generator_matrix=GM,bs=n_test)
testing = False
verbose = True
def full_loss_(X, B):
    return (0.5 / X.size()[0]) * torch.square(X - X @ B).sum()
true_order = projection_order1(B)
optimal_test_loss = full_loss_(X_test, B)
main_mask = torch.eye(d).to(device) == 0
D0 = torch.zeros(d,d).to(device) * main_mask
D1 = torch.zeros(d,d).to(device) * main_mask
true_distance = (D0-B).norm()
distance = torch.ceil(true_distance)*2
starting_loss = full_loss_(X_test, D0)


In [3]:
def full_loss_(X, B):
    return (0.5 / X.size()[0]) * torch.square(X - X @ B).sum()
true_order = projection_order1(B)
optimal_test_loss = full_loss_(X_test, B)
main_mask = torch.eye(d).to(device) == 0
D0 = torch.zeros(d,d).to(device) * main_mask
D1 = torch.zeros(d,d).to(device) * main_mask
true_distance = (D0-B).norm()
distance = torch.ceil(true_distance)*2
starting_loss = full_loss_(X_test, D0)


In [4]:
main_mask = torch.eye(d).to(device) == 0
D0 = torch.zeros(d,d).to(device) * main_mask
D1 = torch.zeros(d,d).to(device) * main_mask
true_distance = (D0-B).norm()
distance = torch.ceil(true_distance)*2
starting_loss = full_loss_(X_test, D0)


In [5]:
starting_loss = full_loss_(X_test, D0)


In [7]:
#weight names
additive='directional weights'
noimportance='no importance'

@torch.no_grad()
def update_importance(elem1, elem2, weight1=1, weight2=1):
    if weight1==0:
        return elem2, 1
    return (elem1 * weight1 + elem2 * weight2)/(weight1+weight2), weight1+weight2

@torch.no_grad()
def full_loss(D):
    return (0.5 / X_test.size()[0]) * torch.square(X_test - X_test @ D).sum()

def loss(D, bs, mask, strategy=noimportance):
    x  = data_generator(GM, bs=bs)
    columns_batch = torch.zeros(d)
    if strategy!=noimportance:
        with torch.no_grad():
            for x_sample in x:
                columns_batch += x_sample**2
    return 0.5/bs  * torch.square(x - x @ (D * mask)).sum(), columns_batch/bs #+ 0.001* masked_D.norm(p=1) + 0.000 * masked_D.norm(p=2) ** 2

def quadratic_optimization(D, optimizer, num_iter, mask, loss=loss, full_loss=full_loss, bs=1, log_iter=100, testing=testing, strategy='normal'):
    losses_inside = []
    columns_output = torch.zeros_like(D)
    with torch.no_grad():
        D *= mask
    for i in range(num_iter):
        if i % log_iter == 0 and testing:
            losses_inside.append(full_loss(D).item())
        def closure():
            optimizer.zero_grad()
            loss_closure, columns_new = loss(D=D,bs=bs,mask=mask, strategy=strategy)
            nonlocal columns_output
            columns_output,_ = update_importance(columns_output, columns_new, weight1=i)
            return loss_closure
        optimizer.step(closure)

    with torch.no_grad():
        D *= mask
    return D, losses_inside, columns_output
    


In [ ]:
import itertools


best_loss = 1e20
best_D = D1.clone()
best_order = None

epochs = 100
num_iter_outer = 2000
num_iter_inner = 2000
bs = 1
testing = False
verbose = False

strategy=noimportance

# ==========================================================================================
# RANDOM ORDER
# 
results_random_order=[]
for rseed in range(1000):
    torch.random.manual_seed(rseed)
    np.random.seed(rseed)
    print('Random order {}/1000'.format(rseed+1))
    importance_columns, number_of_samples = torch.zeros(d,d).to(device), 0
    D = D0.clone().detach().requires_grad_()
    optimizer = opt.UniversalSGD([D], D=distance)
    main_losses = [full_loss(D).item()]

    order=torch.randperm(d)    #####
    mask=mask_from_order(order, main_mask) ####
    for j in tqdm(range(epochs)):
        D, loss, columns_new = quadratic_optimization(D,optimizer,num_iter_inner, mask=mask, testing=testing, strategy=strategy)

        with torch.no_grad():
            loss2 = full_loss(D * mask).item()
            if verbose:
                print('after', loss2)
            main_losses.append(loss2)
            if loss2 < best_loss:
                best_loss = loss2 + 0.
                best_D = D.detach().clone()
                best_order = order + 0
    results_random_order.append({'strategy':strategy, 'seed':seed, 'main_losses':main_losses, 'D':D})
    
# ==========================================================================================
# CORRECT ORDER
# 
rseed=0
torch.random.manual_seed(rseed)
np.random.seed(rseed)
print('Correct order')
importance_columns, number_of_samples = torch.zeros(d,d).to(device), 0
D = D0.clone().detach().requires_grad_()
optimizer = opt.UniversalSGD([D], D=distance)
main_losses = [full_loss(D).item()]

order=true_order    #####
mask=mask_from_order(order, main_mask) ####
for j in tqdm(range(epochs)):
    D, loss, columns_new = quadratic_optimization(D,optimizer,num_iter_inner, mask=mask, testing=testing, strategy=strategy)

    with torch.no_grad():
        loss2 = full_loss(D * mask).item()
        if verbose:
            print('after', loss2)
        main_losses.append(loss2)
        if loss2 < best_loss:
            best_loss = loss2 + 0.
            best_D = D.detach().clone()
            best_order = order + 0
results_true_order={'strategy':strategy, 'seed':seed, 'main_losses':main_losses, 'D':D}
    

In [ ]:
for result in [results_random_order[0]]:
        plt.semilogy(torch.tensor(result['main_losses'])-optimal_test_loss, label='Random order, seeds 0-99')
plt.semilogy(torch.tensor(results_true_order['main_losses'])-optimal_test_loss, label='Correct order', marker='*', markevery=20)
for result in results_random_order[1:100]:
        plt.semilogy(torch.tensor(result['main_losses'])-optimal_test_loss)

plt.ylabel(r' $f(x_k)-f(\bar{x})$')
plt.xlabel('Epoch')
plt.legend()
plt.savefig('plot_random_order.pdf')


In [ ]:
import pylab as pl

plt.hist([torch.tensor(result['main_losses'][-1])-optimal_test_loss for result in results_random_order], label='Random order')
pl.hist([results_true_order['main_losses'][-1]-optimal_test_loss], bins=np.logspace(-2,1, 50), label='Correct order')
pl.gca().set_xscale("log")
plt.ylabel('Count')
plt.xlabel('Suboptimality after 100 SGD epochs')
plt.title('Vertices: {}'.format(d))
plt.legend()
plt.savefig('histogram_random_order.pdf')



In [ ]:
# import pickle


# with open('random_orders.pickle', 'wb') as handle:
#     pickle.dump((results_random_order, results_true_order), handle, protocol=pickle.HIGHEST_PROTOCOL)


# with open('random_orders.pickle', 'rb') as handle:
#     results_random_order, results_true_order = pickle.load(handle)